We re-create numerical artefacts in the implementation of GPR with a periodic kernel for forward prediction. 
We consider a simple truth trajectory given by a deterministic sine curve. Next, we configure the GPRP algorithm and plot predictions. 

To get started, I import some python software I will need:

In [1]:
import sys
sys.path.append('../../../')
from plot_tools.fig_preamble import * # imports numpy, matplotlib, Py2PDF, plot_helper_funcs, plot_figstyle_sheet
import GPy


from gpr.common import get_data, simple_unlearnable_sine
from data_tools.load_raw_cluster_data import LoadExperiment as le
from plot_tools.plot_figstyle_sheet import color_pallete, predmarker, COLOURDICT, STYLEDICT
from analysis_tools.common import sqr_err

/home/riddhisw/anaconda2/lib/python2.7/site-packages/matplotlib/lines.py:1206: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self._markerfacecolor != fc:
/home/riddhisw/anaconda2/lib/python2.7/site-packages/matplotlib/lines.py:1182: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self._markeredgecolor != ec:


I also set up filepaths for saving data

In [2]:
ver=0
datapath = '../../DATA_v'+str(ver)+'_/' #v'+str(ver)+'/DATA_v'+str(ver)+'/'
datafile = 'GPR_BR_calc_RBF_RQ_MAT32_QPER_PER'
test_case=19
variation=5

In [3]:
dataobject = le(test_case, variation, 
                skip = 1,
                GPRP_load='No', GPRP_path = './',
                LKFFB_load = 'Yes', LKFFB_path = datapath,
                AKF_load='No', AKF_path = './',
                LSF_load = 'No', LSF_path = './')

n_train = dataobject.Expt.n_train
Delta_T = dataobject.Expt.Delta_T_Sampling
n_predict = dataobject.Expt.n_predict
n_testbefore = dataobject.Expt.n_testbefore

LKFFB: Data Loaded? Yes


In [4]:
print dataobject.Expt.n_train, dataobject.Expt.Delta_T_Sampling, dataobject.Expt.n_predict, dataobject.Expt.n_testbefore

2000 0.001 100 50


In [5]:
msmt_var = 1000

sigma_0=1000.
period_0 =4000
length_scale_0=0.001

rbf_lengthscale=10.
rbf_variance=1000.
mat_variance=100.
rq_variance=1000.

qrbf_lengthscale=20.
qrbf_variance=50.
qper_lengthscale=0.001
qper_variance=50.
qper_period=4000

In [6]:
# Set initial starting values for L-BFGS-B OPtimiser in GPy
total_runs=50
kernel_name = ['RBF', 'RQ', 'MAT32', 'QPER', 'PER']
total_opt_params = 50 # total number of Gpy Optimisation params

We now implement optimised GPRP model but with constrained periodicities

In [7]:
pred_list= np.zeros((len(kernel_name), total_runs, n_predict+n_testbefore))
truth_list= np.zeros((len(kernel_name), total_runs, n_predict+n_testbefore))
failed_runs_list = np.zeros((len(kernel_name), total_runs))
opt_params=[]

In [8]:
for idx_run in xrange(total_runs): 

    X, Y, TestX, truth, msmts = get_data(dataobject, points=n_train, randomize='y')
  
    for idx_ker in xrange(len(kernel_name)):        
        
        # Reset
        KERNEL=0
        gauss=0
        exact=0
        m1=0

        # Built model
        KERNEL = {'RBF': GPy.kern.RBF(1, 
                                      lengthscale=rbf_lengthscale,
                                      variance=rbf_variance), 
                  'RQ': GPy.kern.RatQuad(1, variance=rq_variance), 
                  'MAT32': GPy.kern.sde_Matern32(1, variance=mat_variance),
                  'QPER': GPy.kern.RBF(1,
                                      lengthscale=qrbf_lengthscale,
                                      variance=qrbf_variance)*GPy.kern.StdPeriodic(1, 
                                                   period=qper_period, 
                                                   lengthscale=qper_lengthscale,
                                                   variance=qper_variance),
                 'PER': GPy.kern.StdPeriodic(1, period=period_0, 
                                              lengthscale=length_scale_0,
                                              variance=sigma_0),}
        
        gauss = GPy.likelihoods.Gaussian(variance=msmt_var)
        exact = GPy.inference.latent_function_inference.ExactGaussianInference()
        
        
        print("Optimise GPR for:", kernel_name[idx_ker], idx_run)
        
        
        try:
            
            m1 = GPy.core.GP(X=X, Y=Y, kernel=KERNEL[kernel_name[idx_ker]], likelihood=gauss, 
                             inference_method=exact)
            print(m1)
            m1.optimize(optimizer=None, messages=False)
            
            pred_list[idx_ker, idx_run, :] = m1.predict(TestX)[0].flatten()
            print("Optimisation complete!") 
            # Cant raise abnormal terminations in LBFGS in Gpy as exceptions - these are incl in analysis
            print(m1)
            #opt_params.append(m1.flattened_parameters)

        except:
            #opt_params.append(np.zeros(6))
            pred_list[idx_ker, idx_run, :] = np.zeros(int(TestX.shape[0]))
            failed_runs_list[idx_ker, idx_run] = idx_run
            print("Unexpected error:", sys.exc_info()[0])
            print("Failed run", idx_run)

        truth_list[idx_ker, idx_run, :] = truth[int(TestX[0]):] # duplications, but downstream code is easier
        
        
        np.savez(datapath+datafile,
                 truth_list=truth_list, 
                 pred_list=pred_list,
                 #opt_params=opt_params, # too big to save. don't know why!
                 failed_runs_list=failed_runs_list,
                 kernel_name=kernel_name,
                 n_train=n_train, n_predict=n_predict, n_testbefore=n_testbefore,
                 msmt_var=msmt_var, 
                 Delta_T=Delta_T, total_runs=total_runs, test_case=test_case, variation=variation )


('Optimise GPR for:', 'RBF', 0)


 ../../../gpr/common.py:66: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future
 ../../../gpr/common.py:67: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future



Name : gp
Objective : 144354.787223
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  gp.                      |   value  |  constraints  |  priors
  rbf.variance             |  1000.0  |      +ve      |        
  rbf.lengthscale          |    10.0  |      +ve      |        
  Gaussian_noise.variance  |  1000.0  |      +ve      |        


 /home/riddhisw/anaconda2/lib/python2.7/site-packages/paramz/transformations.py:109: RuntimeWarning:overflow encountered in expm1


Optimisation complete!

Name : gp
Objective : 14189.0475676
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  gp.                      |          value  |  constraints  |  priors
  rbf.variance             |  722526.442019  |      +ve      |        
  rbf.lengthscale          |  8.20017743891  |      +ve      |        
  Gaussian_noise.variance  |  51324.5182267  |      +ve      |        
('Optimise GPR for:', 'RQ', 0)

Name : gp
Objective : 185026.888032
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                      |   value  |  constraints  |  priors
  RatQuad.variance         |  1000.0  |      +ve      |        
  RatQuad.lengthscale      |     1.0  |      +ve      |        
  RatQuad.power            |     2.0  |      +ve      |        
  Gaussian_noise.variance  |  1000.0  |      +ve      |        
Optimisation complete!

Name : gp
Objective : 14189.0483933
Number of Parameters : 4
Num

...

Optimisation complete!

Name : gp
Objective : 14681.0916989
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |            value  |  constraints  |  priors
  std_periodic.variance     |    107389.356312  |      +ve      |        
  std_periodic.period       |     4000.1268833  |      +ve      |        
  std_periodic.lengthscale  |  0.0049241028822  |      +ve      |        
  Gaussian_noise.variance   |     80739.891229  |      +ve      |        
('Optimise GPR for:', 'RBF', 5)

Name : gp
Objective : 158656.86687
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  gp.                      |   value  |  constraints  |  priors
  rbf.variance             |  1000.0  |      +ve      |        
  rbf.lengthscale          |    10.0  |      +ve      |        
  Gaussian_noise.variance  |  1000.0  |      +ve      |        
Optimisation complete!

Name : gp
Objective : 14254.8221015
Number

 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/standard_periodic.py:112: RuntimeWarning:overflow encountered in divide
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/standard_periodic.py:113: RuntimeWarning:invalid value encountered in sin


Optimisation complete!

Name : gp
Objective : 5029.47796497
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |               value  |  constraints  |  priors
  std_periodic.variance     |       1316645.39316  |      +ve      |        
  std_periodic.period       |       4590.00480987  |      +ve      |        
  std_periodic.lengthscale  |   2.97517784882e-07  |      +ve      |        
  Gaussian_noise.variance   |  5.56268464627e-309  |      +ve      |        
('Optimise GPR for:', 'RBF', 29)

Name : gp
Objective : 158804.475147
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  gp.                      |   value  |  constraints  |  priors
  rbf.variance             |  1000.0  |      +ve      |        
  rbf.lengthscale          |    10.0  |      +ve      |        
  Gaussian_noise.variance  |  1000.0  |      +ve      |        
Optimisation complete!

Name : gp
Objective : 141

 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/standard_periodic.py:134: RuntimeWarning:overflow encountered in divide


('Unexpected error:', <class 'numpy.linalg.linalg.LinAlgError'>)
('Failed run', 39)
('Optimise GPR for:', 'PER', 39)

Name : gp
Objective : 190613.077435
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |   value  |  constraints  |  priors
  std_periodic.variance     |  1000.0  |      +ve      |        
  std_periodic.period       |  4000.0  |      +ve      |        
  std_periodic.lengthscale  |   0.001  |      +ve      |        
  Gaussian_noise.variance   |  1000.0  |      +ve      |        
Optimisation complete!

Name : gp
Objective : 14706.2176072
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |             value  |  constraints  |  priors
  std_periodic.variance     |     93412.3041275  |      +ve      |        
  std_periodic.period       |     4000.06195729  |      +ve      |        
  std_periodic.lengthscale  |  0.00523139898008  |      

In [9]:
opt_params

[]

In [10]:
pred_list.shape

(5, 50, 150)

In [11]:
fs

NameError: name 'fs' is not defined